In [32]:
import pandas as pd
import numpy as np
import random
from nltk.tokenize import RegexpTokenizer

from nltk.stem.wordnet import WordNetLemmatizer

from gensim.corpora import Dictionary
import pickle as pkl

import guidedlda
from sklearn.feature_extraction.text import CountVectorizer

In [48]:
# get the clearned reviews 
with open("clean_reviews.pkl", "rb") as file:
    clean_reviews = pkl.load(file)
    print(clean_reviews[0])

['dr', 'goldberg', 'offer', 'everything', 'look', 'for', 'in', 'general', 'practitioner', 'he', 'nice', 'and', 'easy', 'to', 'talk', 'to', 'without', 'being', 'patronizing', 'he', 'always', 'on', 'time', 'in', 'seeing', 'his', 'patient', 'he', 'affiliated', 'with', 'top', 'notch', 'hospital', 'nyu', 'which', 'my', 'parent', 'have', 'explained', 'to', 'me', 'is', 'very', 'important', 'in', 'case', 'something', 'happens', 'and', 'you', 'need', 'surgery', 'and', 'you', 'can', 'get', 'referral', 'to', 'see', 'specialist', 'without', 'having', 'to', 'see', 'him', 'first', 'really', 'what', 'more', 'do', 'you', 'need', 'sitting', 'here', 'trying', 'to', 'think', 'of', 'any', 'complaint', 'have', 'about', 'him', 'but', 'really', 'drawing', 'blank']


In [ ]:
# with open("clean_review_corpus.pkl", "rb") as file:
#     corpus = pkl.load(file)
#     print(corpus[0])

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 2), (20, 1), (21, 3), (22, 1), (23, 2), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 2), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 2), (40, 1), (41, 2), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 1), (50, 1), (51, 1), (52, 1), (53, 1), (54, 1), (55, 2), (56, 3)]


#### Downsample Reviews for Checkpoint

In [ ]:
print(len(clean_reviews))
random.seed(10701)
clean_reviews_down = random.sample(clean_reviews, 10000)
print(len(clean_reviews_down))

700000
10000


In [ ]:
clean_reviews_down[0]

['this',
 'wa',
 'the',
 'first',
 'place',
 'in',
 'vega',
 'where',
 'the',
 'yelpers',
 'let',
 'me',
 'down',
 'the',
 'salt',
 'and',
 'pepper',
 'shrimp',
 'appetizer',
 'wa',
 'inedible',
 'all',
 'of',
 'the',
 'shell',
 'wa',
 'left',
 'on',
 'underneath',
 'the',
 'crispy',
 'fried',
 'coating',
 'a',
 'result',
 'the',
 'sauce',
 'used',
 'to',
 'marinate',
 'the',
 'shrimp',
 'never',
 'penetrated',
 'the',
 'meat',
 'it',
 'coagulated',
 'in',
 'the',
 'head',
 'and',
 'exploded',
 'on',
 'you',
 'upon',
 'taking',
 'bite',
 'gross',
 'the',
 'potstickers',
 'and',
 'the',
 'beef',
 'chow',
 'fun',
 'lacked',
 'any',
 'flavor',
 'whatsoever',
 'the',
 'only',
 'redeeming',
 'quality',
 'the',
 'service',
 'is',
 'pretty',
 'awful',
 'is',
 'the',
 'roast',
 'duck',
 'crispy',
 'and',
 'tasty',
 'skin',
 'with',
 'relatively',
 'moist',
 'meat',
 'and',
 'relative',
 'bargain',
 'compared',
 'to',
 'place',
 'in',
 'ny',
 'and']

In [ ]:
# get the vocab_list
dictionary = Dictionary(clean_reviews_down)
dictionary.filter_extremes(no_below=20, no_above=0.5)
vocab_list = list(dictionary.token2id.keys())

INFO:gensim.corpora.dictionary:adding document #0 to Dictionary<0 unique tokens: []>
INFO:gensim.corpora.dictionary:built Dictionary<28589 unique tokens: ['a', 'all', 'and', 'any', 'appetizer']...> from 10000 documents (total 1251924 corpus positions)
INFO:gensim.utils:Dictionary lifecycle event {'msg': "built Dictionary<28589 unique tokens: ['a', 'all', 'and', 'any', 'appetizer']...> from 10000 documents (total 1251924 corpus positions)", 'datetime': '2024-11-19T11:35:47.712755', 'gensim': '4.3.3', 'python': '3.10.14 (main, May  6 2024, 14:42:37) [Clang 14.0.6 ]', 'platform': 'macOS-14.7-arm64-arm-64bit', 'event': 'created'}
INFO:gensim.corpora.dictionary:discarding 25200 tokens: [('and', 8815), ('coagulated', 1), ('coating', 12), ('exploded', 3), ('in', 6244), ('is', 6213), ('it', 6913), ('marinate', 2), ('of', 6719), ('penetrated', 1)]...
INFO:gensim.corpora.dictionary:keeping 3389 tokens which were in no less than 20 and no more than 5000 (=50.0%) documents
INFO:gensim.corpora.dict

In [ ]:
# with open("vocabulary.pkl", "wb") as file:
#     pkl.dump(vocab_list, file)

In [ ]:
# with open("vocabulary.pkl", "rb") as file:
#     vocab_list = pkl.load(file)

In [38]:
def create_matrix(processed_text, vocabulary=vocab_list):
    vectorizer = CountVectorizer(vocabulary=vocabulary)
    dtm = vectorizer.fit_transform(processed_text)
    return dtm, vectorizer.vocabulary_

In [ ]:
processed_corpus = [" ".join(tokens) for tokens in clean_reviews_down]
dtm, vocab_dict = create_matrix(processed_corpus)


# Hyperparameters and Seeds

In [54]:
# Hyperparameters
N_TOPIC = 25            # number of clusters
N_ITER = 500            # number of iterations run on dataset
RANDOM_STATE = 7        # initial random seed
REFRESH = 20            # how often you print log-likelihood
SEED_CONFIDENCE=0.15    # probability of using this word as a seed

# create topic seeds
# seed_topic_list = [
#     ["medicine", "office", "hurt"],
#     ["coffee", "pizza", "delicious"]
# ]
seed_topic_list = [
    ["pizza", "food", "chicken", "burger"],
    ["doctor", "office", "medical"]
]

seed_topics = {}
for topic_id, topic_words in enumerate(seed_topic_list):
    for word in topic_words:
        seed_topics[vocab_dict[word]] = topic_id


# Training/Fitting Model

In [55]:
# building the model
model = guidedlda.GuidedLDA(n_topics=N_TOPIC, n_iter=N_ITER, random_state=RANDOM_STATE, refresh=REFRESH)

In [56]:
# fitting the model
model.fit(dtm, seed_topics=seed_topics, seed_confidence=SEED_CONFIDENCE)
# model.fit(dtm, seed_confidence=SEED_CONFIDENCE)

INFO:guidedlda:n_documents: 10000
INFO:guidedlda:vocab_size: 3389
INFO:guidedlda:n_words: 872500
INFO:guidedlda:n_topics: 25
INFO:guidedlda:n_iter: 500
INFO:guidedlda:<0> log likelihood: -9980162
INFO:guidedlda:<20> log likelihood: -7004175
INFO:guidedlda:<40> log likelihood: -6845304
INFO:guidedlda:<60> log likelihood: -6780632
INFO:guidedlda:<80> log likelihood: -6740308
INFO:guidedlda:<100> log likelihood: -6711140
INFO:guidedlda:<120> log likelihood: -6686449
INFO:guidedlda:<140> log likelihood: -6666613
INFO:guidedlda:<160> log likelihood: -6654162
INFO:guidedlda:<180> log likelihood: -6641791
INFO:guidedlda:<200> log likelihood: -6631873
INFO:guidedlda:<220> log likelihood: -6625680
INFO:guidedlda:<240> log likelihood: -6621122
INFO:guidedlda:<260> log likelihood: -6614256
INFO:guidedlda:<280> log likelihood: -6608415
INFO:guidedlda:<300> log likelihood: -6607059
INFO:guidedlda:<320> log likelihood: -6603700
INFO:guidedlda:<340> log likelihood: -6602095
INFO:guidedlda:<360> log l

## Top Words per Topic

In [57]:
topic_word = model.topic_word_
print(len(topic_word))
n_top_words = 25
for i, topic_dist in enumerate(topic_word):
    # print(topic_dist)
    topic_words = np.array(vocab_list)[np.argsort(topic_dist)][:-(n_top_words+1):-1]
    print('Topic {}: {}'.format(i, ' '.join(topic_words)))

25
Topic 0: pizza burger good with fry my they had cheese not on sandwich place have food like chicken so just are very were all sauce salad
Topic 1: my she me he her so not had on said just back when what like up have there with at would one asked out did
Topic 2: room hotel at with not stay on vega are nice pool night casino very strip bed you from had stayed there here floor were which
Topic 3: de le la u00e9 et n1 n2 est n3 u00e0 un que pa da en die n4 u00e9e on pour du une plus au service
Topic 4: with were had we my on dessert which salad steak cream meal menu wine cheese so all delicious restaurant our good very an bread made
Topic 5: food not had with restaurant were good dish very sauce my nthe menu salad just like be chicken on would or they star service flavor
Topic 6: food good roll chicken rice with place sushi my soup not thai very fried had spicy noodle they chinese dish on here restaurant like sauce
Topic 7: my they me not on with have had be at he them time an no when 

## Find Topic given Review

In [ ]:
dtm_transform = model.transform(dtm)

dtm_topics = np.argmax(dtm_transform, axis=1)

(array([   7,   14,   26,   55,   58,   66,   86,  143,  149,  167,  176,
        177,  184,  190,  210,  219,  240,  242,  279,  301,  311,  320,
        321,  329,  331,  333,  339,  341,  376,  382,  388,  395,  401,
        404,  414,  428,  429,  456,  511,  512,  516,  527,  571,  572,
        580,  590,  609,  614,  631,  648,  661,  683,  689,  691,  697,
        700,  721,  736,  760,  783,  787,  798,  801,  834,  872,  883,
        909,  957,  979,  988, 1002, 1019, 1030, 1096, 1102, 1105, 1124,
       1143, 1155, 1156, 1158, 1159, 1185, 1192, 1210, 1219, 1236, 1243,
       1250, 1253, 1255, 1259, 1272, 1289, 1291, 1303, 1310, 1319, 1323,
       1350, 1358, 1359, 1381, 1411, 1418, 1439, 1489, 1495, 1497, 1515,
       1522, 1555, 1614, 1636, 1638, 1641, 1652, 1745, 1831, 1838, 1856,
       1861, 1865, 1882, 1896, 1897, 1927, 1931, 1945, 1946, 1962, 1968,
       1969, 1977, 1989, 2000, 2006, 2015, 2034, 2035, 2056, 2068, 2090,
       2092, 2105, 2114, 2118, 2140, 2154, 2177, 2

In [71]:
vocab_map = np.vectorize(lambda x: vocab_list[x])

for topic_idx in range(N_TOPIC):
    # Get 5 example documents
    topic_idx_docs = np.where(dtm_topics == topic_idx)[0][:5]
    print(topic_idx_docs)
    doc_dtm = dtm[topic_idx_docs].toarray()
    print(doc_dtm)
    # Get 15 terms from each document
    common_terms = np.argsort(doc_dtm, axis=1)[:, ::-1][:,:15]
    print(common_terms)
    print(vocab_map(common_terms))
    document_words = vocab_map(common_terms)
    for doc in document_words:
        print(" ".join(doc))
    break

[ 7 14 26 55 58]
[[0 0 0 ... 0 0 0]
 [0 2 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[[ 220  395  399  407  406  423  421  110  396  412  400  415  411  413
   414]
 [ 372    1  142  683  140   28  685  392  684  119  678  199  682  681
    89]
 [  52  411  963  904  409  966  603  962  964  965  967  266  968  969
   902]
 [ 142  372   89  834   52  397 1408  362  268   29  202   98  413 1193
   574]
 [ 220   52  834  372   92  351   28  143 1430  912   27 1244 1295 1180
  1432]]
[['good' 'beer' 'brunch' 'flatbread' 'dog' 'patio' 'opted' 'my' 'biscuit'
  'gravy' 'calling' 'grilled' 'fry' 'great' 'greatest']
 ['pizza' 'all' 'we' 'own' 'very' 'on' 'saw' 'than' 'premium' 'one'
  'agreed' 'ever' 'mixed' 'ingredient' 'had']
 ['with' 'fry' 'lunch' 'sub' 'french' 'reasonable' 'nothing' 'free'
  'mention' 'presented' 'recently' 'or' 'soda' 'turkey' 'run']
 ['we' 'pizza' 'had' 'crust' 'with' 'bread' 'delivered' 'cheese'
  'ordered' 'only' 'extra' 'into' 'great' 'rec

In [25]:
# Select 1000 reviews
item_term_matrix, _ = create_matrix(processed_corpus[:1000])
item_term_matrix

<1000x31595 sparse matrix of type '<class 'numpy.int64'>'
	with 75714 stored elements in Compressed Sparse Row format>

In [ ]:
doc_topic = model.transform(item_term_matrix)

top_topic = np.argmax(doc_topic, axis=1) #find the topic of the document
print(top_topic)

for topic_idx in range(N_TOPIC):
    # Grab 5 example documents from each topic
    topic_idx_docs = np.where(top_topic == topic_idx)[:5]
    doc_dtm = item_term_matrix[topic_idx_docs].toarray()
    

topic_doc_indices = np.where(top_topic == 0)[0][:5] #find documents from topic 0
print(topic_doc_indices)
print(item_term_matrix[topic_doc_indices].toarray()) #print document term matrix for those documents
print(np.argsort(item_term_matrix[topic_doc_indices].toarray(), axis=1)[:,::-1]) #argsort to find the most common terms
print(np.argsort(item_term_matrix[topic_doc_indices].toarray(), axis=1)[:,::-1][:, :5]) #same thing, gives top 5

vectorized_vocab_map = np.vectorize(lambda x: vocab_list[x])

print(vectorized_vocab_map(np.argsort(item_term_matrix[topic_doc_indices].toarray(), axis=1)[:,::-1][:, :5])) #get words corresponding to indices

for i in range(len(doc_topic)):  # Use the correct range based on your data
    # Get the top topic for the document
    top_topic = doc_topic[i].argmax()
    
    # Get the top terms more frequent than 1 in the document from the dtm
    row = item_term_matrix[i, :].toarray().flatten()  # Convert sparse row to dense
    # item_count = np.count_nonzero(row > 1)
    top_terms_indices = row.argsort()[::-1][:15]  # Indices of the top 10 terms
    top_terms = [vocab_list[idx] for idx in top_terms_indices]
    
    # Print the result
    print(f"Top topic: {top_topic}, Document terms: {', '.join(top_terms)}")

[20 20  6 20  0  8  8 24  1 24 14 24  6  6  2  8 24 19  1 19 19 19 19 23
 13  6 17 18 20 20  1  0  3  4 17 15 15 15 15  2 15 17 18  9 13 15  0 15
 20  1]
[ 4 31 46]
[[1 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[[  323   332   169 ... 21069 21070 15797]
 [ 1666   554   190 ... 21069 21070     0]
 [ 1137   243   612 ... 21069 21070     0]]
[[ 323  332  169  338   70]
 [1666  554  190  171  586]
 [1137  243  612  169 2057]]
[['hot' 'sauce' 'are' 'wing' 'good']
 ['chinese' 'food' 'even' 'away' 'taste']
 ['soup' 'best' 'around' 'are' 'chow']]
Top topic: 20, Document terms: he, you, him, need, really, have, see, without, parent, referral, patient, on, patronizing, offer, practitioner
Top topic: 20, Document terms: he, all, very, been, over, your, dr, patient, year, out, think, health, option, question, one
Top topic: 6, Document terms: you, office, he, doctor, not, when, about, away, they, me, call, johnson, will, before, practice
Top topic: 20, Document terms: have, dr, doctor,

In [ ]:
# TODO: Try making code to extract 5 documents per topic

## Saving the Model

In [ ]:
# prunes matrices, saves weights but locks model
# model.purge_extra_matrices()

In [ ]:
# modelname = f"model_{N_TOPIC}_{N_ITER}_{RANDOM_STATE}_{REFRESH}_{SEED_CONFIDENCE}.pkl"

# with open(f"results/{modelname}", 'wb') as file:
    # pkl.dump(model, file)

In [ ]:
# Checking if the model can be loaded/works
# with open('guidedlda_model.pickle', 'rb') as file:
    # model = pkl.load(file)
# model.transform(create_matrix(processed_corpus[:1])[0])

array([[3.23365735e-04, 1.55314518e-03, 4.57218647e-04, 2.87699878e-04,
        1.87135438e-03, 2.22783475e-04, 2.12591695e-01, 2.84587882e-04,
        5.12376868e-02, 5.04084413e-04, 1.43815767e-03, 3.04138392e-04,
        3.15682987e-06, 3.29554255e-02, 1.25188044e-02, 3.53338186e-04,
        6.78863908e-06, 3.08103614e-04, 4.92289652e-03, 4.52324088e-04,
        6.72737138e-01, 2.55559447e-03, 1.49780723e-03, 3.40117465e-04,
        2.72587228e-04]])

In [24]:
model.loglikelihoods_

[-768290932.3669106,
 -544845646.4478563,
 -523167504.4543283,
 -517464505.4454495,
 -515242153.73157394,
 -514171247.29473853,
 -513573261.2712008,
 -513116435.8723612]

# Save Results

Make sure to run this after each experiment (after you've created/fitted the model)

In [72]:
filename = f"result_{N_TOPIC}_{N_ITER}_{RANDOM_STATE}_{REFRESH}_{SEED_CONFIDENCE}.txt"

with open(f"results/{filename}", "w") as f:
    if seed_topic_list:
        print(seed_topic_list, file=f)
    else:
        print("No Seeds", file=f)
        
    dtm_transform = model.transform(dtm)
    dtm_topics = np.argmax(dtm_transform, axis=1)
    vocab_map = np.vectorize(lambda x: vocab_list[x])


    # Save top 25 words per topic
    topic_word = model.topic_word_
    n_top_words = 25
    for idx, topic_dist in enumerate(topic_word):
        topic_words = np.array(vocab_list)[np.argsort(topic_dist)][:-(n_top_words+1):-1]
        print('Topic {}: {}'.format(idx, ' '.join(topic_words)), file=f)
        print("Example Documents:", file=f)
        
        # Get 5 example documents
        topic_idx_docs = np.where(dtm_topics == i)[0][:5]
        
        doc_dtm = dtm[topic_idx_docs].toarray()
        
        # Get 15 terms from each document
        common_terms = np.argsort(doc_dtm, axis=1)[:, ::-1][:,:15]
        document_words = vocab_map(common_terms)
        for doc in document_words:
            print(" ".join(doc), file=f)
        
        # Newline as separator
        print(file=f)
    

print("Saved in", filename)



Saved in result_25_500_7_20_0.15.txt
